# Main() and command-line arguments

The Main method is the entry point of a C# application. (Libraries and services do not require a Main method as an entry point.) When the application is started, the Main method is the first method that is invoked.

There can only be one entry point in a C# program. If you have more than one class that has a Main method, you must compile your program with the StartupObject compiler option to specify which Main method to use as the entry point. For more information, see <a href="https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/compiler-options/advanced#mainentrypoint-or-startupobject"> StartupObject (C# Compiler Options)</a>.

In [ ]:
class TestClass
{
    static void Main(string[] args)
    {
        // Display the number of command line arguments.
        Console.WriteLine(args.Length);
    }
}

Starting in C# 9, you can omit the Main method, and write C# statements as if they were in the Main method, as in the following example:

In [ ]:
using System.Text;

StringBuilder builder = new();
builder.AppendLine("Hello");
builder.AppendLine("World!");

Console.WriteLine(builder.ToString());

Hello
World!



For information about how to write application code with an implicit entry point method, 
see <a href="https://docs.microsoft.com/en-us/dotnet/csharp/fundamentals/program-structure/top-level-statements">
 Top-level statements.</a>

## Overview
* The Main method is the entry point of an executable program; it is where the program control starts and ends.
* Main is declared inside a class or struct. Main must be static and it need not be public. (In the earlier example, it receives the default access of private.) The enclosing class or struct is not required to be static.
* Main can either have a void, int, or, starting with C# 7.1, Task, or Task<int> return type.
* If and only if Main returns a Task or Task<int>, the declaration of Main may include the async modifier. This specifically excludes an async void Main method.
* The Main method can be declared with or without a string[] parameter that contains command-line arguments. When using Visual Studio to create Windows applications, you can add the parameter manually or else use the GetCommandLineArgs() method to obtain the command-line arguments. Parameters are read as zero-indexed command-line arguments. Unlike C and C++, the name of the program is not treated as the first command-line argument in the args array, but it is the first element of the GetCommandLineArgs() method.
The following list shows valid Main signatures:

In [ ]:
// public static void Main() { }   // no use of args or await
// public static int Main() { }    // no use of  args or await
// public static void Main(string[] args) { }  // uses args, no use await 
// public static int Main(string[] args) { }   // uses args, no use await 
// public static async Task Main() { }             // no use args, uses await
// public static async Task<int> Main() { }        // no use args, uses await
// public static async Task Main(string[] args) { }        // uses args and await
// public static async Task<int> Main(string[] args) { }   // uses args and await

## Main() return values
You can return an int from the Main method by defining the method in one of the following ways:

<table>
    <tr>
        <th>Main method code</th>	
        <th>Main signature</th>
    </tr>
    <tr>
        <td>No use of args or await</td>
        <td>static int Main()</td>
    </tr>
    <tr>
        <td>Uses args, no use of await</td>
        <td>static int Main(string[] args)</td>
    </tr>
    <tr>
        <td>No use of args, uses await</td>
        <td>static async Task< int > Main()</td>
    </tr>
    <tr>
        <td>Uses args and await</td>
        <td>static async Task< int > Main(string[] args)</td>
    </tr>
</table>
<p>If the return value from Main is not used, returning void or Task allows for slightly simpler code.</p>
<table>
    <tr>
        <th>Main method code</th>	
        <th>Main signature</th>
    </tr>
    <tr>
        <td>No use of args or await</td>
        <td>static void Main()</td>
    </tr>
    <tr>
        <td>Uses args, no use of await</td>
        <td>static void Main(string[] args)</td>
    </tr>
    <tr>
        <td>No use of args, uses await</td>
        <td>static async Task Main()</td>
    </tr>
    <tr>
        <td>Uses args and await</td>
        <td>static async Task Main(string[] args)</td>
    </tr>
</table>


However, returning int or Task<int> enables the program to communicate status information to other programs or scripts that invoke the executable file.

The following example 
shows
 how the exit code for the process can be accessed.

This example uses .NET Core command-line tools. If you are unfamiliar with .NET Core command-line tools, you can learn about them in this get-started article.

Create a new application by running dotnet new console. Modify the Main method in Program.cs as follows:

In [ ]:
// Save this program as MainReturnValTest.cs.
class MainReturnValTest
{
    static int Main()
    {
        //...
        return 0;
    }
}

When a program is executed in Windows, any value returned from the Main function is stored in an environment variable. This environment variable can be retrieved using ERRORLEVEL from a batch file, or $LastExitCode from PowerShell.

You can build the application 
using
 the dotnet CLI dotnet build command.

Next, create a PowerShell script to run the application and display the result. Paste the following code into a text file and save it as test.ps1 in the folder that contains the project. Run the PowerShell script by typing test.ps1 at the PowerShell prompt.

Because the code 
returns
 zero, the batch file will report success. However, if you change MainReturnValTest.cs to return a non-zero value and then recompile the program, subsequent execution of the PowerShell script will report failure.

## Async Main return values
When you declare an async return value for Main, the compiler generates the boilerplate code for calling asynchronous methods in Main. If you don't specify the async keyword, you need to write that code yourself, as shown in the following example. The code in the example ensures that your program runs until the asynchronous operation is completed:

In [ ]:
public static void Main()
{
    AsyncConsoleWork().GetAwaiter().GetResult();
}

private static async Task<int> AsyncConsoleWork()
{
    // Main body here
    return 0;
}

This boilerplate code can be replaced by:

In [ ]:
static async Task<int> Main(string[] args)
{
    return await AsyncConsoleWork();
}

An advantage of declaring Main as async is that the compiler always generates the correct code.

When the application entry point returns a Task or Task<int>, the compiler generates a new entry point that calls the entry point method declared in the application code. Assuming that this entry point is called $GeneratedMain, the compiler generates the following code for these entry points:

* static Task Main() results in the compiler emitting the equivalent of private static void $GeneratedMain() => Main().GetAwaiter().GetResult();
* static Task Main(string[]) results in the compiler emitting the equivalent of private static void $GeneratedMain(string[] args) => Main(args).GetAwaiter().GetResult();
* static Task<int> Main() results in the compiler emitting the equivalent of private static int $GeneratedMain() => Main().GetAwaiter().GetResult();
* static Task<int> Main(string[]) results in the compiler emitting the equivalent of private static int $GeneratedMain(string[] args) => Main(args).GetAwaiter().GetResult();

#### Note:
If the examples used async modifier on the Main method, the compiler would generate the same code.

#### Note 
You can also use <b>Environment.CommandLine</b> or <b>Environment.GetCommandLineArgs</b> to access the command-line arguments from any point in a console or Windows Forms application. To enable command-line arguments in the Main method signature in a Windows Forms application, you must manually modify the signature of Main. The code generated by the Windows Forms designer creates Main without an input parameter.

The parameter of the Main method is a String array that represents the command-line arguments. Usually you determine whether arguments exist by testing the Length property, for example:

In [ ]:
if (args.Length == 0)
{
    System.Console.WriteLine("Please enter a numeric argument.");
    return 1;
}

Error: (1,5): error CS0103: The name 'args' does not exist in the current context

#### Tip
The args array can't be null. So, it's safe to access the Length property without null checking.

You can also convert the string arguments to numeric types by using the Convert class or the Parse method. For example, the following statement converts the string to a long number by using the Parse method:

In [ ]:
long num = Int64.Parse(args[0]);

//It is also possible to use the C# type long, which aliases Int64:

long num2 = long.Parse(args[0]);

// You can also use the Convert class method ToInt64 to do the same thing:

long num3 = Convert.ToInt64(s);

Error: (1,24): error CS0103: The name 'args' does not exist in the current context
(5,24): error CS0103: The name 'args' does not exist in the current context
(9,29): error CS0103: The name 's' does not exist in the current context

For more information, 
see
 <a href="https://docs.microsoft.com/en-us/dotnet/api/system.int64.parse">Parse</a> and <a href="https://docs.microsoft.com/en-us/dotnet/api/system.convert">Convert.</a>

The following example shows how to use command-line arguments in a console application. The application takes one argument at run time, converts the argument to an integer, and calculates the factorial of the number. If no arguments are supplied, the application issues a message that explains the correct usage of the program.

To compile and run the application from a command prompt, follow these steps:

* Paste the following code into any text editor, and then save the file as a text file with the name Factorial.cs.

In [ ]:
public class Functions
{
    public static long Factorial(int n)
    {
        // Test for invalid input.
        if ((n < 0) || (n > 20))
        {
            return -1;
        }

        // Calculate the factorial iteratively rather than recursively.
        long tempResult = 1;
        for (int i = 1; i <= n; i++)
        {
            tempResult *= i;
        }
        return tempResult;
    }
}

class MainClass
{
    static int Main(string[] args)
    {
        // Test if input arguments were supplied.
        if (args.Length == 0)
        {
            Console.WriteLine("Please enter a numeric argument.");
            Console.WriteLine("Usage: Factorial <num>");
            return 1;
        }

        // Try to convert the input arguments to numbers. This will throw
        // an exception if the argument is not a number.
        // num = int.Parse(args[0]);
        int num;
        bool test = int.TryParse(args[0], out num);
        if (!test)
        {
            Console.WriteLine("Please enter a numeric argument.");
            Console.WriteLine("Usage: Factorial <num>");
            return 1;
        }

        // Calculate factorial.
        long result = Functions.Factorial(num);

        // Print result.
        if (result == -1)
            Console.WriteLine("Input must be >= 0 and <= 20.");
        else
            Console.WriteLine($"The Factorial of {num} is {result}.");

        return 0;
    }
}
// If 3 is entered on command line, the
// output reads: The factorial of 3 is 6.